3.ipynb：一份PDF文档，有三个（问题，答案）对 - 进阶RAG（优化后的检索器 + 生成器优化） - Ragas评估

- 3.1.ipynb：优化后的检索器 + 基于LLMChainExtractor的文本压缩器 = 上下文压缩

# 加载环境变量

In [1]:
import os
from dotenv import find_dotenv, load_dotenv

# 加载环境变量
load_dotenv(find_dotenv())

True

# 源数据

[2023全球智能汽车AI挑战赛——赛道一：AI大模型检索问答](https://tianchi.aliyun.com/competition/entrance/532154/forum)

# 构建RAG

In [2]:
def pretty_print_docs(docs):
  print(
      f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n + {d.page_content}\n{d.metadata}" for i,d in enumerate(docs)])
  )

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# 加载文档
file_path = "data/初赛训练数据集.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
print(len(docs))

354


In [4]:
pretty_print_docs(docs[:3])

Document 1:

 + 欢迎
感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。
首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。
本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际
接收的车辆为准。
如您有任何问题，或需要预约服务，请拨打电话4006-010101 联系我们。您也可以开车前往Lynk &Co领克中心。
在抵达之前，请您注意驾车安全。©领克汽车销售有限公司
{'source': 'data/初赛训练数据集.pdf', 'page': 0}
----------------------------------------------------------------------------------------------------
Document 2:

 + 
{'source': 'data/初赛训练数据集.pdf', 'page': 1}
----------------------------------------------------------------------------------------------------
Document 3:

 + 3目录
前言
本手册相关的重要信息 .................................................11
敬告用户.................................................................11
联系Lynk&Co领克 .....................................................12
事件数据记录系统 ......................................................12
远程监控系统............................................................12
原厂精装附件、选装装备和改装 ......................................13
无线电设备 ..................

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 分割文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50,
)
split_docs = text_splitter.split_documents(docs)
print(len(split_docs))

807


In [6]:
pretty_print_docs(split_docs[:3])

Document 1:

 + 欢迎
感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。
首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。
本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际
接收的车辆为准。
如您有任何问题，或需要预约服务，请拨打电话4006-010101 联系我们。您也可以开车前往Lynk &Co领克中心。
在抵达之前，请您注意驾车安全。©领克汽车销售有限公司
{'source': 'data/初赛训练数据集.pdf', 'page': 0}
----------------------------------------------------------------------------------------------------
Document 2:

 + 3目录
前言
本手册相关的重要信息 .................................................11
敬告用户.................................................................11
联系Lynk&Co领克 .....................................................12
{'source': 'data/初赛训练数据集.pdf', 'page': 2}
----------------------------------------------------------------------------------------------------
Document 3:

 + 事件数据记录系统 ......................................................12
远程监控系统............................................................12
原厂精装附件、选装装备和改装 ......................................13
{'source': 'data/初赛训练数据集.

In [7]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

# 创建嵌入模型
model_name = 'BAAI/bge-large-zh-v1.5'
model_kwargs = {'device': 'cuda'}  # 需要安装GPU版本的torch 
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_community.vectorstores import FAISS

# 创建向量存储
vectordb = FAISS.from_documents(split_docs, embeddings)

In [9]:
index_folder_path = "data/faiss_index"
index_name = "3.1"

In [10]:
# 保存索引
vectordb.save_local(index_folder_path, index_name)

In [11]:
# 加载索引
vectordb = FAISS.load_local(index_folder_path, embeddings, index_name)

In [12]:
# 创建密集检索器，擅长根据语义相似度查找相关文档
faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [13]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# 创建稀疏检索器，擅长根据关键词查找相关文档
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k=10

# 创建混合检索器
retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever], 
                              weight=[0.5, 0.5])

测试

In [14]:
# question = "怎么打开危险警告灯？"
# retrieval_docs = retriever.invoke(question)
# len(retrieval_docs)

In [15]:
# # 去重后
# len(list(set([doc.page_content for doc in retrieval_docs])))

In [16]:
# pretty_print_docs(retrieval_docs)

In [17]:
# pretty_print_docs(faiss_retriever.invoke(question))

In [18]:
# pretty_print_docs(bm25_retriever.invoke(question))

In [19]:
from langchain_openai import ChatOpenAI

# 创建模型
llm = ChatOpenAI(temperature=0)
print(llm.model_name)

gpt-3.5-turbo


[LLMChainExtractor文档](https://api.python.langchain.com/en/v0.0.354/retrievers/langchain.retrievers.document_compressors.chain_extract.LLMChainExtractor.html#langchain.retrievers.document_compressors.chain_extract.LLMChainExtractor)

[LLMChainExtractor类代码](https://github.com/langchain-ai/langchain/blob/v0.0.354/libs/langchain/langchain/retrievers/document_compressors/chain_extract.py#L47)

[LLMChainExtractor类prompt](https://github.com/langchain-ai/langchain/blob/v0.0.354/libs/langchain/langchain/retrievers/document_compressors/chain_extract_prompt.py)

```python
prompt_template = """Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return {no_output_str}. 

Remember, *DO NOT* edit the extracted parts of the context.

> Question: {{question}}
> Context:
>>>
{{context}}
>>>
Extracted relevant 
```parts:"""

In [20]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# 创建文档压缩器：LLMChainExtractor ，它将遍历最初返回的文档，并仅从每个文档中提取与查询相关的内容。
compressor = LLMChainExtractor.from_llm(llm)

# 创建上下文压缩检索器：需要传入一个文档压缩器和基本检索器
# 上下文压缩检索器将查询传递到基本检索器，获取初始文档并将它们传递到文档压缩器。
# 文档压缩器获取文档列表，并通过减少文档内容或完全删除文档来缩短文档列表。
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [21]:
from langchain.chains import RetrievalQA

# 创建链
chain = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=compression_retriever,
                                 return_source_documents=True)

In [22]:
# 运行链 - 测试
question = "怎么打开危险警告灯？"
response = chain(question)
print(response)

D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method

{'query': '怎么打开危险警告灯？', 'result': '要打开危险警告灯，只需按下车辆上的危险警告灯按键即可。按下按键后，车辆的两侧转向指示灯会开始闪烁，表示危险警告灯已经启用。', 'source_documents': [Document(page_content='使用危险警告灯', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 3}), Document(page_content='危险警告灯按键\n车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危\n险警告灯。', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 88}), Document(page_content='打开危险警告灯开关时，两侧转向指示灯均闪烁。', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 73})]}


In [23]:
response['query']

'怎么打开危险警告灯？'

In [24]:
response['result']

'要打开危险警告灯，只需按下车辆上的危险警告灯按键即可。按下按键后，车辆的两侧转向指示灯会开始闪烁，表示危险警告灯已经启用。'

In [25]:
pretty_print_docs(response['source_documents'])

Document 1:

 + 使用危险警告灯
{'source': 'data/初赛训练数据集.pdf', 'page': 3}
----------------------------------------------------------------------------------------------------
Document 2:

 + 危险警告灯按键
车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危
险警告灯。
{'source': 'data/初赛训练数据集.pdf', 'page': 88}
----------------------------------------------------------------------------------------------------
Document 3:

 + 打开危险警告灯开关时，两侧转向指示灯均闪烁。
{'source': 'data/初赛训练数据集.pdf', 'page': 73}


# 评估RAG

[使用您的测试集进行评估](https://docs.ragas.io/en/stable/getstarted/evaluation.html)

数据集包含以下列：

- question: list[str] - 这些是将评估您的 RAG 管道的问题。
- answer: list[str] - 您的 RAG 管道生成的答案。
- context: list[list[str]] - 传递到 LLM 来回答问题的上下文。
- ground_truth: list[str] - 问题的真实答案。


In [26]:
questions = ["怎么打开危险警告灯？",
            "车辆如何保养？",
            "靠背太热怎么办？"]

ground_truths = ["危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。",
            "为了保持车辆处于最佳状态，建议您定期关注车辆状态，包括定期保养、洗车、内部清洁、外部清洁、轮胎的保养、低压蓄电池的保养等。",
            "您好，如果您的座椅靠背太热，可以尝试关闭座椅加热功能。在多媒体显示屏上依次点击空调开启按键→座椅→加热，在该界面下可以关闭座椅加热。"]

In [27]:
# 生成 answers 和 contexts
answers = []
contexts = []

for question in questions:
    print(question)
    response = chain(question)
    print(response['result'], "\n")
    answers.append(response['result'])
    contexts.append([doc.page_content for doc in response['source_documents']])

怎么打开危险警告灯？


D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method

要打开危险警告灯，只需按下车辆上的危险警告灯按键即可。这样可以启用车辆的危险警告灯，用于在车辆遇到交通事故或其他紧急情况时向其他驾驶员发出警告。 

车辆如何保养？


D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method

车辆保养包括多个方面，其中之一是保养轮胎。轮胎对车辆的驾驶特性有很大影响，因此保持轮胎的良好状态至关重要。以下是一些保养轮胎的建议：
- 定期检查轮胎是否损坏，如割伤或裂缝，以及不定期检查是否存在异常磨损。
- 注意避免让轮胎接触到润滑脂、润滑油和燃油。
- 保持正确的充气压力和良好的驾驶习惯，这有助于延长轮胎的使用寿命。
- 定期清洗轮胎，确保轮胎表面干净，避免污垢对轮胎的影响。

除了轮胎保养外，车辆保养还包括定期清洗车辆外表、检查车漆损伤并修复、清洗车底等。保养车辆可以延长车辆的寿命，提高安全性能和驾驶舒适度。 

靠背太热怎么办？


D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method

您可以尝试以下方法来解决靠背太热的问题：
1. 调整座椅的角度，让靠背离开身体，增加通风空间。
2. 使用透气性好的座椅套或垫，帮助吸收汗水并保持通风。
3. 在长时间坐车前，可以提前打开车窗通风，让车内空气流通。
4. 如果可能的话，停车时可以将车辆停在阴凉处，避免长时间暴晒。
希望这些方法能帮助到您！ 



In [28]:
# 构建评估数据集
from datasets import Dataset

evaluate_data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

evaluate_dataset = Dataset.from_dict(evaluate_data)
evaluate_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 3
})

In [29]:
evaluate_dataset[0]

{'question': '怎么打开危险警告灯？',
 'answer': '要打开危险警告灯，只需按下车辆上的危险警告灯按键即可。这样可以启用车辆的危险警告灯，用于在车辆遇到交通事故或其他紧急情况时向其他驾驶员发出警告。',
 'contexts': ['使用危险警告灯',
  '使用危险警告灯\n危险警告灯按键\n车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危\n险警告灯。',
  '打开危险警告灯'],
 'ground_truth': '危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。'}


[评估指标](https://docs.ragas.io/en/stable/concepts/metrics/index.html)
- Faithfulness
  - 如果答案中提出的所有主张都可以从给定的上下文中推断出来，则生成的答案被认为是忠实的。
  - answer 和 contexts
- Answer Relevancy
  - 评估指标“答案相关性”重点评估生成的答案与给定提示的相关程度。不完整或包含冗余信息的答案将获得较低分数。
  - answer 和 question
- Context Precision
  - 用于评估 contexts 中存在的所有真实相关项目是否排名较高。理想情况下，所有相关块必须出现在顶层。
  - question 和 contexts
- Context Recall
  - 上下文回忆衡量检索到的上下文与带注释的答案（被视为基本事实）的一致程度。
  - ground truth 和 contexts
---

- RAGAS评估框架评估了RAG管道的两个主要组件：
    - Retriever 检索器
    - Generator 生成器

---

- 与评估 Retrieval 相关的指标如下：
    - Context Precision。question 和 contexts
    - Context Recall。ground truth 和 contexts
- 与评估 Generation 相关的指标如下：
  - Faithfulness。answer 和 contexts
  - Answer Relevancy。answer 和 question

In [30]:
# 导入指标
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


# 开始评估
from ragas import evaluate

evaluate_result = evaluate(
    evaluate_dataset, 
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ]
)
evaluate_result

Evaluating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:37<00:00,  3.11s/it]


{'faithfulness': 1.0000, 'answer_relevancy': 0.9479, 'context_recall': 1.0000, 'context_precision': 0.4556}

In [31]:
df_evaluate_result = evaluate_result.to_pandas()
df_evaluate_result

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision
0,怎么打开危险警告灯？,要打开危险警告灯，只需按下车辆上的危险警告灯按键即可。这样可以启用车辆的危险警告灯，用于在车...,"[使用危险警告灯, 使用危险警告灯\n危险警告灯按键\n车辆遇到交通事故或其他紧急情况时，按...",危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。,1.0,0.958339,1.0,1.000000
1,车辆如何保养？,车辆保养包括多个方面，其中之一是保养轮胎。轮胎对车辆的驾驶特性有很大影响，因此保持轮胎的良好...,[保养项目\n保养时间\n按照时间和里程确定保养周期。正常情况下，车辆的行驶里程决定了\n执...,为了保持车辆处于最佳状态，建议您定期关注车辆状态，包括定期保养、洗车、内部清洁、外部清洁、轮...,1.0,0.938730,1.0,0.366667
2,靠背太热怎么办？,您可以尝试以下方法来解决靠背太热的问题：\n1. 调整座椅的角度，让靠背离开身体，增加通风空...,[],您好，如果您的座椅靠背太热，可以尝试关闭座椅加热功能。在多媒体显示屏上依次点击空调开启按键→...,1.0,0.946748,1.0,0.000000


In [32]:
df_evaluate_result.to_excel(f'data/{index_name}_eval_result.xlsx', index=False)